In [1]:
# ---------------------------------------------------------
# ZELLE 1: SETUP & SPARK SESSION
# ---------------------------------------------------------
# Hier initialisieren wir den "Driver".
# Wir nutzen Spark im "Local Mode" (master="local[*]"), um einen
# Cluster auf diesem Rechner zu simulieren.
# Außerdem definieren wir relative Pfade, damit das Notebook bei jedem im Team läuft.
# ---------------------------------------------------------

import os
import shutil
import findspark

findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, from_unixtime
from pyspark.sql.types import DoubleType, LongType, StringType

spark = SparkSession.builder \
    .appName("OpenSky Flight Data Analysis") \
    .master("local[*]") \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .getOrCreate()

current_dir = os.getcwd()
RAW_PATH = os.path.join(current_dir, "..", "data", "raw")
PROCESSED_PATH = os.path.join(current_dir, "..", "data", "processed")

print(f"Spark Version: {spark.version}")
print(f"Lese Rohdaten aus: {RAW_PATH}")

Spark Version: 3.5.7
Lese Rohdaten aus: c:\Users\Luca-\Desktop\FlyDataRepo\fly_big_data\notebooks\..\data\raw


In [2]:
# ---------------------------------------------------------
# ZELLE 2: PHASE 1 - INGESTION (RAW ZONE / BRONZE LAYER)
# ---------------------------------------------------------
# Wir laden die Rohdaten (CSV) aus dem "Data Lake".
#
# Herausforderung: Die Daten liegen teilweise in versteckten Ordnern (beginnend mit .),
# die Spark standardmäßig ignoriert.
# Lösung: Wir suchen die Dateipfade erst mit Python (os.walk) rekursiv und
# übergeben Spark dann die exakte Liste der Dateien.
#
# Strategie: "Schema-on-Read" -> Spark versucht, Datentypen automatisch zu erkennen.
# ---------------------------------------------------------

import os

# 1. Pfade robust setzen (egal ob aus VS Code Root oder Notebook-Ordner gestartet)
current_work_dir = os.getcwd()
if os.path.exists(os.path.join(current_work_dir, "data", "raw")):
    RAW_PATH = os.path.join(current_work_dir, "data", "raw")
else:
    # Fallback: Wir sind im 'notebooks' Unterordner
    RAW_PATH = os.path.abspath(os.path.join(current_work_dir, "..", "data", "raw"))

print(f"Lese Daten aus: {RAW_PATH}")

# 2. Dateien manuell einsammeln (Workaround für versteckte Ordner)
csv_files = []
if os.path.exists(RAW_PATH):
    for root, dirs, files in os.walk(RAW_PATH):
        for file in files:
            if file.endswith(".csv"):
                csv_files.append(os.path.join(root, file))
    print(f"-> {len(csv_files)} CSV-Dateien gefunden.")
else:
    print(f"FEHLER: Ordner {RAW_PATH} nicht gefunden.")

# 3. Spark Ingestion
if len(csv_files) > 0:
    # Wir übergeben die Liste der Dateien direkt an Spark
    df_raw = spark.read \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .csv(csv_files)

    print("\nSchema der Rohdaten (automatisch erkannt):")
    df_raw.printSchema()
    
else:
    print("!!! WARNUNG: Keine Daten gefunden. Bitte Download-Skript prüfen.")

Lese Daten aus: c:\Users\Luca-\Desktop\FlyDataRepo\fly_big_data\data\raw
-> 5 CSV-Dateien gefunden.

Schema der Rohdaten (automatisch erkannt):
root
 |-- time: integer (nullable = true)
 |-- icao24: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- velocity: double (nullable = true)
 |-- heading: double (nullable = true)
 |-- vertrate: double (nullable = true)
 |-- callsign: string (nullable = true)
 |-- onground: boolean (nullable = true)
 |-- alert: boolean (nullable = true)
 |-- spi: boolean (nullable = true)
 |-- squawk: integer (nullable = true)
 |-- baroaltitude: double (nullable = true)
 |-- geoaltitude: double (nullable = true)
 |-- lastposupdate: double (nullable = true)
 |-- lastcontact: double (nullable = true)



In [3]:
# ---------------------------------------------------------
# ZELLE 3: PHASE 2 - PROCESSING & STORAGE (SILVER LAYER)
# ---------------------------------------------------------
# Dies ist der Kern des Data Engineering (ETL):
# 1. Cleaning: Casting von Strings zu korrekten Datentypen (Double/Long).
# 2. Filter: Entfernen von Datensätzen ohne Geokoordinaten.
# 3. Storage Optimization: Wir speichern die Daten als PARQUET.
# ---------------------------------------------------------

df_cleaned = df_raw.select(
    col("time").cast(LongType()),
    col("callsign").cast(StringType()),
    col("lat").cast(DoubleType()),
    col("lon").cast(DoubleType()),
    col("velocity").cast(DoubleType()),
    col("geoaltitude").cast(DoubleType())
)
df_cleaned = df_cleaned.filter(col("lat").isNotNull() & col("lon").isNotNull())

df_silver = df_cleaned.withColumn("flight_date", to_date(from_unixtime(col("time"))))

print(f"Schreibe optimierte Daten nach: {PROCESSED_PATH} ...")
df_silver.write \
    .mode("overwrite") \
    .partitionBy("flight_date") \
    .parquet(PROCESSED_PATH)

print("ETL Erfolgreich. Daten sind jetzt im Silver Layer.")

Schreibe optimierte Daten nach: c:\Users\Luca-\Desktop\FlyDataRepo\fly_big_data\notebooks\..\data\processed ...
ETL Erfolgreich. Daten sind jetzt im Silver Layer.


In [ ]:
# ---------------------------------------------------------
# ZELLE 4: PHASE 3 - ANALYTICS (GOLD LAYER)
# ---------------------------------------------------------
# Ab hier arbeiten wir nur noch mit den optimierten Parquet-Daten (Silver Layer).
#
# Schritte für den "Gold Layer" (Business-Level Daten):
# 1. Data Quality: Wir filtern "technischen Müll" heraus (NULL-Werte,
#    leere Strings, Test-Daten). Das ist entscheidend für saubere Reports.
# 2. Aggregation (OLAP): Wir gruppieren nach Flugzeugen und zählen die Wegpunkte.
#    -> Performance-Vorteil: Dank Parquet (Column-oriented) muss Spark hierfür
#       nur die Spalte 'callsign' von der Festplatte lesen.
# 3. Selection: Wir wählen automatisch den Flug mit den meisten Datenpunkten
#    für die nachfolgende Visualisierung aus.
# ---------------------------------------------------------

import folium
import pandas as pd
from pyspark.sql.functions import col, desc, trim, length

# 1. Laden aus dem Silver Layer (Parquet)
df_analytics = spark.read.parquet(PROCESSED_PATH)

# --- Robustes Cleaning (Data Quality) ---
# Wir säubern die Callsigns, um Geister-Flüge zu vermeiden:
# - trim: Entfernt Leerzeichen am Anfang/Ende
# - isNotNull: Entfernt technische NULLs
# - length > 3: Entfernt unvollständige Kürzel
# - != TEST...: Entfernt Testdaten
df_valid_flights = df_analytics.filter(
    col("callsign").isNotNull() & 
    (trim(col("callsign")) != "") & 
    (col("callsign") != "TEST1234") &
    (length(trim(col("callsign"))) > 3)
)

# 2. Aggregation: Top 5 ECHTE Airlines finden
top_flights = df_valid_flights.groupBy("callsign") \
    .count() \
    .orderBy(col("count").desc())

print("Top 5 saubere Callsigns (nach Anzahl Wegpunkten):")
top_flights.show(5)

# 3. Kandidaten für die Karte auswählen
# Wir nehmen automatisch den ersten (aktivsten) Flug aus der bereinigten Liste
target_row = top_flights.first()

if target_row:
    target_callsign = target_row["callsign"]
    count = target_row["count"]
    print(f"Wir visualisieren gleich Flug: '{target_callsign}' mit {count} Wegpunkten.")
else:
    print("Keine validen Flüge gefunden.")

Top 5 saubere Callsigns:
+--------+-----+
|callsign|count|
+--------+-----+
|POZARNI4| 1793|
|SWP1    | 1784|
|UAE924  | 1772|
|A6EOU   | 1770|
|ASA773  | 1728|
+--------+-----+
only showing top 5 rows

✅ Wir visualisieren jetzt Flug: 'POZARNI4' mit 1793 Wegpunkten.


In [ ]:
# ---------------------------------------------------------
# ZELLE 5: PHASE 4 - VISUALISIERUNG (SERVING LAYER)
# ---------------------------------------------------------
# Schnittstelle zwischen "Big Data" (Spark) und "Small Data" (Python/Pandas).
# 1. Wir filtern auf EINEN spezifischen Flug (Reduktion der Datenmenge).
# 2. Wir nutzen .toPandas(), um die Daten vom Cluster-Speicher in den lokalen RAM zu holen.
# 3. Visualisierung der Route mittels Folium (interaktive Karte).
# ---------------------------------------------------------

import folium
import pandas as pd

flight_trace = df_analytics \
    .filter(col("callsign") == target_callsign) \
    .select("lat", "lon", "geoaltitude", "time") \
    .orderBy("time") \
    .limit(500) \
    .toPandas()

if not flight_trace.empty:
    center_lat = flight_trace["lat"].mean()
    center_lon = flight_trace["lon"].mean()
    
    m = folium.Map(location=[center_lat, center_lon], zoom_start=6)

    route_points = list(zip(flight_trace["lat"], flight_trace["lon"]))
    
    folium.PolyLine(route_points, color="red", weight=3, opacity=0.8).add_to(m)
    
    folium.Marker(route_points[0], popup="Start", icon=folium.Icon(color="green")).add_to(m)
    folium.Marker(route_points[-1], popup="Ende", icon=folium.Icon(color="blue")).add_to(m)
    
    display(m)
else:
    print("Keine Daten gefunden.")